# Exploratory Data Analysis

In [1]:
# Import packages
import pandas as pd
import os

## Functions

In [2]:
def load_data(path_to_file, sample = 5):
    
    """
    Load a csv file into a dataframe and display a brief overview of the first 5 rows.
    """
    
    if not os.path.exists(path_to_file):
        raise FileNotFoundError(f"The file '{path_to_file}' does not exist.")
    
    # Load client data into a datafram
    df = pd.read_csv(path_to_file)
    
    # Display for a brief overview
    display(df.head(sample))
    display(df.info())

    return df

def clean_datatype(df, float_columns = None, int_columns = None, str_columns = None, date_columns = None, bool_columns = None):
    
    """
    Converts specified columns in the dataframe to the desired data types. 
    
    Requirement: 
    - No missing values
    
    Parameters:
    - float_columns: List of column names to convert to float
    - int_columns: List of column names to convert to int
    - str_columns: List of column names to convert to string
    - date_columns: List of column names to convert to datetime
    - bool_columns: List of column names containing boolean-like strings ('t', 'true', 'f', 'false') to convert to int
    
    Returns:
    - Dataframe with updated column types
    """

    # Default empty lists if no list is provided
    int_columns = int_columns or []
    str_columns = str_columns or []
    date_columns = date_columns or []
    bool_columns = bool_columns or []
    
    if float_columns:
        for col in float_columns:
            df[col] = df[col].astype(float)

    if int_columns:
        for col in int_columns:
            df[col] = df[col].astype(int)
    
    if str_columns:
        for col in str_columns:
            df[col] = df[col].astype(str)
    
    if date_columns:
        for col in date_columns:
            df[col] = pd.to_datetime(df[col]) 
            # Adjust the datetime conversion here if the format looks different
    
    if bool_columns:
        # Convert true and false to integer
        for col in bool_columns:
            df[col] = df[col].apply(lambda x: 1 if str(x).lower() in ['t', 'true'] else 0)

    return df

def clean_channel_sales(df):

    """
    Converts unconvenient sales channel names to integer dummies with respect to their frequency
    """

    # Remove rows belonging to channels with value counts below 20 can be removed because they don't have much impact on the final model
    valid_channels = df.channel_sales.value_counts()[df.channel_sales.value_counts() > 20].index
    df = df[df.channel_sales.isin(valid_channels)]

    # Rename remaining channels for simplicity according to their rank in frequency
    frequency_map = df.channel_sales.value_counts().rank(method='dense', ascending=False).astype(int) - 1
    df.loc[:, 'channel_sales'] = df.channel_sales.map(frequency_map)
                                                            
    return df

## Client Data

In [3]:
# Load the data
client_df = load_data('../data/raw/client_data.csv')

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,t,0.00,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,f,0.00,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,f,0.00,28.60,28.60,1,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,f,0.00,30.22,30.22,1,25.46,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,f,52.32,44.91,44.91,1,47.98,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,19.800,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14606 entries, 0 to 14605
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              14606 non-null  object 
 1   channel_sales                   14606 non-null  object 
 2   cons_12m                        14606 non-null  int64  
 3   cons_gas_12m                    14606 non-null  int64  
 4   cons_last_month                 14606 non-null  int64  
 5   date_activ                      14606 non-null  object 
 6   date_end                        14606 non-null  object 
 7   date_modif_prod                 14606 non-null  object 
 8   date_renewal                    14606 non-null  object 
 9   forecast_cons_12m               14606 non-null  float64
 10  forecast_cons_year              14606 non-null  int64  
 11  forecast_discount_energy        14606 non-null  float64
 12  forecast_meter_rent_12m         

None

In [4]:
# Treat possible missing values here

In [5]:
# Convert channel sales to dummy variables
client_df = clean_channel_sales(client_df)

# Clean the datatypes of the columns
client_df = clean_datatype(client_df, 
               float_columns = ['cons_12m', 'cons_gas_12m', 'cons_last_month', 'forecast_cons_year'], 
               date_columns = ['date_activ', 'date_end', 'date_modif_prod', 'date_renewal'],
               bool_columns = ['has_gas'],
               int_columns = ['channel_sales'])
client_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14590 entries, 0 to 14605
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              14590 non-null  object        
 1   channel_sales                   14590 non-null  int64         
 2   cons_12m                        14590 non-null  float64       
 3   cons_gas_12m                    14590 non-null  float64       
 4   cons_last_month                 14590 non-null  float64       
 5   date_activ                      14590 non-null  datetime64[ns]
 6   date_end                        14590 non-null  datetime64[ns]
 7   date_modif_prod                 14590 non-null  datetime64[ns]
 8   date_renewal                    14590 non-null  datetime64[ns]
 9   forecast_cons_12m               14590 non-null  float64       
 10  forecast_cons_year              14590 non-null  float64       
 11  forecas

In [12]:
# Save as new csv file
client_df.to_csv('../data/clean/client_data.csv', index = False)

## Price Data

In [7]:
# Load the data
price_df = load_data('../data/raw/price_data.csv')

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193002 entries, 0 to 193001
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  193002 non-null  object 
 1   price_date          193002 non-null  object 
 2   price_off_peak_var  193002 non-null  float64
 3   price_peak_var      193002 non-null  float64
 4   price_mid_peak_var  193002 non-null  float64
 5   price_off_peak_fix  193002 non-null  float64
 6   price_peak_fix      193002 non-null  float64
 7   price_mid_peak_fix  193002 non-null  float64
dtypes: float64(6), object(2)
memory usage: 11.8+ MB


None

In [8]:
# Treat possible missing values here

In [9]:
# Clean the datatypes of the columns
price_df = clean_datatype(price_df, date_columns = ['price_date'])
price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193002 entries, 0 to 193001
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   id                  193002 non-null  object        
 1   price_date          193002 non-null  datetime64[ns]
 2   price_off_peak_var  193002 non-null  float64       
 3   price_peak_var      193002 non-null  float64       
 4   price_mid_peak_var  193002 non-null  float64       
 5   price_off_peak_fix  193002 non-null  float64       
 6   price_peak_fix      193002 non-null  float64       
 7   price_mid_peak_fix  193002 non-null  float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 11.8+ MB


In [10]:
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [13]:
# Save as new csv file
price_df.to_csv('../data/clean/price_data.csv', index = False)